In [1]:
import math
import nltk
from datetime import datetime
from nltk.corpus import wordnet
nltk.download('words')
nltk.download('stopwords')
corpus = set(nltk.corpus.words.words())
stopwords = set(nltk.corpus.stopwords.words())

import pandas as pd
import numpy as np
import regex as re
import wbdata
from easymoney.money import EasyPeasy

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def get_summary(dat):

    dtypes_dict = dat.dtypes.apply(lambda x: x.name).to_dict()

    numeric = [x for x in dtypes_dict.keys() if dtypes_dict[x] == 'float64' or dtypes_dict[x] == 'int64']
    chars = [x for x in dtypes_dict.keys() if dtypes_dict[x] == 'object']

    type_numeric = ['numeric'] * len(numeric)
    type_chars = ['char'] * len(chars)

    numeric_counts = dat.loc[:, numeric].count().to_dict()
    numeric_populated = list(numeric_counts.values())
    numeric_pct = [str((round(x / len(dat), 2)) * 100) + '%' for x in numeric_populated]
    numeric_zero = [sum(dat.loc[:, x] == 0) for x in numeric]

    numeric_s = pd.DataFrame(list(zip(numeric, type_numeric, numeric_populated, numeric_pct, numeric_zero)),
                             columns=['Name', 'Type', 'Non_NA_Values', 'Pct_Non_NA', 'Num_Zeros'])

    numeric_cols_df = pd.concat([numeric_s,
                                 dat.describe().transpose().reset_index().drop(['count', 'index'], axis=1)],
                                axis=1)

    char_counts = dat.loc[:, chars].count().to_dict()
    char_populated = list(char_counts.values())
    char_pct = [str((round(x / len(dat), 3)) * 100) + '%' for x in char_populated]
    char_zero = [sum(dat.loc[:, x] == 0) for x in chars]

    chars_unique = [len(dat.loc[:, x].unique()) for x in chars]
    chars_val_max = [dat.loc[:, x].value_counts().idxmax() for x in chars]

    chars_df = pd.DataFrame(
        list(zip(chars, type_chars, char_populated, char_pct, char_zero, chars_unique, chars_val_max)),
        columns=['Name', 'Type', 'Non_NA_Values', 'Pct_Non_NA', 'Num_Zeros', 'Unique', 'Most_Common'])

    which = input('Would you like a table of numeric or char columns? (N/C)')
    if which == 'N':
        return numeric_cols_df
    elif which == 'C':
        return chars_df

# Comments

In [3]:
comments = pd.concat([pd.read_csv('F:/capstone_listenfirst/10kplus000', delimiter='\t', header=None),
                      pd.read_csv('F:/capstone_listenfirst/1k-10k000', delimiter='\t', header=None)])

C:\Users\Alex\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
cols = ['comment_id', 'comment_dcs_uid', 'comment_author_id', 'post_dcs_uid', 'post_author_dcs_uid', 
        'comment_lfm_type', 'comment_posted_at', 'comment_message', 'comment_sentiment_polarity',
        'comment_primary_emotion', 'comment_ace_metadata', 'topics', 'comment_is_quote', 'spec_channel' ]
comments.columns = cols

In [8]:
def comment_cleaner(text):
    text = re.sub("[^\w\s]", "", text)
    text = " ".join([x.lower() for x in text.split(' ') if x.lower() in corpus and x.lower() not in stopwords and len(x)>1])
    if text == '':
        return np.nan
    return text

In [11]:
comments = comments[comments['comment_message'].notna()]
comments['comments_clean'] = comments['comment_message'].apply(comment_cleaner)

<ipython-input-11-b57297315d5f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['comments_clean'] = comments['comment_message'].apply(comment_cleaner)


In [16]:
comments.to_csv('F:/capstone_listenfirst/comments.concat.csv', index=False, mode='a')

# Deltas

In [564]:
delt1 = pd.read_csv('F:/capstone_listenfirst/facebook_post_deltas_1k-10k.tsv')
delt2 = pd.read_csv('F:/capstone_listenfirst/facebook_post_deltas_10kplus.tsv')

delt = pd.concat([delt1, delt2], ignore_index=True)

KeyboardInterrupt: 

In [ ]:
delt.to_csv('F:/capstone_listenfirst/facebook_post_deltas_concat.csv')

# Brand

## Concat 1-10k, 10k Plus Data

In [47]:
dat1 = pd.read_csv('F:/capstone_listenfirst/brand_source_affiliations_1k-10k.tsv', delimiter='\t')
dat2 = pd.read_csv('F:/capstone_listenfirst/brand_source_affiliations_10kplus.tsv', delimiter='\t')

dat = pd.concat([dat1, dat2], ignore_index=True)

In [48]:
extras = pd.read_csv('F:/capstone_listenfirst/brand_names_extras.csv')
extras['production_company'] = extras['production_company'].str.rstrip()

In [49]:
bo = pd.read_csv('../data/movie_data_w_bo_fixed.csv')

## Create two columns for genre: top_genre and genre_list.

Top genre takes the genre mentioned first (or the only genre) for each movie

Genre list is a repeat of the genres column but in list form

In [50]:
dat['genre_list'] = [x.split(', ') for x in dat['genres'].values]
dat['top_genre'] = [x[0] for x in dat['genre_list'].values]

## Create release year column and release year bins

In [51]:
dat['released_year'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y') if pd.notnull(x) else '' for x in dat['released_on']]
dat['released_year'] = dat['released_year'].astype(str)
dat['year_bin'] = pd.qcut(pd.to_datetime(dat['released_on'], format='%Y-%m-%d').dt.year, q=5, labels=['1', '2', '3', '4', '5'])

In [52]:
dat.to_csv('F:/capstone_listenfirst/brand_concat.csv')

In [53]:
dat

,brand_ods_id,replace,released_on,ticker_symbol,genres,data_profile_dcs_uid,data_profile_channel_type,data_profile_source_type,data_affiliation_active_flag,data_affiliation_start_date_key,data_affiliation_end_date_key,genre_list,top_genre,released_year,year_bin
0,41301,Blade Runner 2049,2017-10-06,T,Sci Fi,45b03b4ab3e6e5781426f83613af0807,instagram,page,t,101,7769,[Sci Fi],Sci Fi,2017,2
1,25859,Geostorm,2017-10-20,T,"Action, Sci Fi",ea5eb96fa2d53c68a7fec4a76636c6ff,twitter,search_term,t,101,7769,"[Action, Sci Fi]",Action,2017,2
2,44895,Ready Player One,2018-03-29,T,"Sci Fi, Action, Thriller",31e9bbe160e77e8558e35bb5c5bd7089,facebook,search_term,t,101,7769,"[Sci Fi, Action, Thriller]",Sci Fi,2018,3
3,10617,Fury,2014-10-17,NaN,"Action, Drama, War",6c7dc49a1cece02cc80d0826e6424df2,facebook,search_term,t,101,7769,"[Action, Drama, War]",Action,2014,1
4,10607,Nightcrawler,2014-10-31,NaN,"Crime, Thriller, Drama",aaec8ee14f2e372b8e35a72ee9613ca1,twitter,search_term,t,101,7769,"[Crime, Thriller, Drama]",Crime,2014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5948,124558,Yesterday,2019-06-28,CMCSA,Comedy,5ed919f3ff31a0e3bf2ac52a3c51e54e,twitter,search_term,t,101,7769,[Comedy],Comedy,2019,4
5949,127224,Brahms: The Boy II,2020-02-21,NaN,"Mystery, Horror, Thriller",915fb763858e8d833ac9bc65ded66671,twitter,search_term,t,101,7769,"[Mystery, Horror, Thriller]",Mystery,2020,5
5950,10553,How to Train Your Dragon 2,2014-06-13,FOXA,"Adventure, Action, Animation",b309e9bf3223aa010a13ff30ae16a09d,twitter,search_term,t,101,7769,"[Adventure, Action, Animation]",Adventure,2014,1
5951,59294,Jurassic World: Fallen Kingdom,2018-06-22,CMCSA,"Adventure, Sci Fi, Action",de2417a5fec251126d3a9ee5bfabfd39,twitter,search_term,t,101,7769,"[Adventure, Sci Fi, Action]",Adventure,2018,3


## Join in extra data and filter

In [54]:
dat['replace'] = [x.rstrip() for x in dat['replace']]
dat = pd.merge(dat, extras, how='inner', left_on='replace', right_on='movies')
dat = dat.rename(columns={'budget ': 'budget', 'rating ': 'rating', 'source ': 'source'})

## Remove extra columns

In [55]:
dat = dat[dat['release_year']!='remove'].drop_duplicates('replace').drop(['movies', 'poc', 'brand_ods_id_y', 'notes'], axis=1)

In [56]:
dat['released_year'] = np.where(dat['released_year']=='', dat['release_year'], dat['released_year'])

## Fill NA values: released_on

In [57]:
fill_dates = dat[dat['released_on'].isna()]
dat = dat[dat['released_on'].notna()]

In [58]:
fill_dates['released_on'] = ['2009-08-07',
                             '1999-07-09',
                             '1988-07-15',
                             'remove',
                             'remove',
                             'remove',
                             '2010-12-03',
                             '2009-01-09',
                             '2004-07-18',
                             '1990-12-07',
                             '2020-01-17',
                             '2010-07-09',
                             '2004-03-09',
                             '1999-10-15',
                             '2009-12-23',
                             '2009-04-03']

In [59]:
dat = dat.append(fill_dates[fill_dates['released_on']!='remove'])
dat = dat.drop('release_year', axis=1)

## Fill NA valaues: production_company

In [60]:
fill_prod = dat[dat['production_company'].isna()]
dat = dat[dat['production_company'].notna()]

In [61]:
fill_prod['production_company'] = ['Amazon Studios',
                                   'remove',
                                   'remove',
                                   'Chartoff Winkler Productions',
                                   'remove',
                                   'Opus Film',
                                   'DreamWorks Pictures']

In [62]:
dat = dat.append(fill_prod[fill_prod['production_company']!='remove'])

## Fill NA values: rating

In [63]:
fill_rating = dat[(dat['rating'].isna()) | (dat['rating']=='Not Rated')]
dat = dat[(dat['rating'].notna()) & (dat['rating']!='Not Rated')]

In [64]:
fill_rating['rating'] = ['remove',
                         'remove', 
                         'remove',
                         'remove', 
                         'remove', 
                         'R', 
                         'R',
                         'remove',
                         'remove',
                         'PG',
                         'PG-13',
                         'R',
                         'PG-13',
                         'R',
                         'R',
                         'G']

In [65]:
dat = dat.append(fill_rating[fill_rating['rating']!='remove'])

## Fill NA values: source

In [66]:
fill_source = dat[dat['source'].isna()]
dat = dat[dat['source'].notna()]

In [67]:
fill_source['source'] = ['Original',
                         'Original']

In [68]:
dat = dat.append(fill_source)

## Add box office

In [69]:
dat = pd.merge(dat, bo, left_on='replace', right_on='Movie')
dat = dat.drop(['Unnamed: 0', 'date URL', 'ReleaseDate', 'summary URL', 'Movie', 'ProductionBudget', 'WorldwideGross'], axis=1)

In [70]:
dat = dat.drop_duplicates('replace')

In [72]:
fill_bo = dat[dat['DomesticGross']==0]
dat = dat[dat['DomesticGross']!=0]

In [76]:
fill_bo['budget'] = [8000000.0,
                     40000000.0,
                     18000000.0]

fill_bo['DomesticGross'] = [517788,
                            3300000,
                            4400000]

In [79]:
dat = dat.append(fill_bo)

In [80]:
dat.to_csv('F:/capstone_listenfirst/brands_with_extras.csv')

## Add extra variables for KModes

In [81]:
ep = EasyPeasy()

In [82]:
dat = dat.reset_index().drop('index', axis=1)

In [83]:
for x, y in enumerate(dat.itertuples()):
    if y[13] in ['Sci Fi', 'Fantasy']:
        dat.at[x, 'genre_grouped'] = 'Fantasy'
    elif y[13] in ['Action']:
        dat.at[x, 'genre_grouped'] = 'Action'
    elif y[13] in ['Crime', 'Thriller', 'Mystery']:
        dat.at[x, 'genre_grouped'] = 'Thriller' 
    elif y[13] in ['Biography', 'Documentary', 'Sport', 'History', 'War']:
        dat.at[x, 'genre_grouped'] = 'Documentary'
    elif y[13] in ['Horror']:
        dat.at[x, 'genre_grouped'] = 'Horror'
    elif y[13] in ['Adventure', 'Western']:
        dat.at[x, 'genre_grouped'] = 'Adventure'
    elif y[13] in ['Romance', 'Drama', 'Adult']:
        dat.at[x, 'genre_grouped'] = 'Drama'
    elif y[13] in ['Comedy']:
        dat.at[x, 'genre_grouped'] = 'Comedy'
    elif y[13] in ['Animation', 'Musical', 'Music', 'Family']:
        dat.at[x, 'genre_grouped'] = 'Family'

In [84]:
dat['released_year_filter'] = pd.to_datetime(dat['released_year'], format='%Y').dt.year.astype(int)

dat = dat[dat['released_year_filter']>=2010]
dat = dat.drop('released_year_filter', axis=1)

In [85]:
dat['budget'] = dat['budget'].fillna(0)
dat['inflated_budget'] = [ep.normalize(amount=x[1]['budget'], region="US", base_currency="USD", from_year=x[1]['released_year'], to_year="latest") for x in dat.iterrows()]
dat['inflated_budget'].astype(int)
dat['budget_cat'] = pd.cut(dat['inflated_budget'], [-5, 6000000, 50000000, 500000000], labels=['LowBudget', 'MediumBudget', 'HighBudget'])

dat['inflated_boxoffice'] = [ep.normalize(amount=x[1]['DomesticGross'], region="US", base_currency="USD", from_year=x[1]['released_year'], to_year="latest") for x in dat.iterrows()]
dat['inflated_boxoffice'].astype(int)

dat['released_year'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y') if pd.notnull(x) else '' for x in dat['released_on']]
dat['released_year'] = dat['released_year'].astype(str)
dat['year_bin'] = pd.cut(pd.to_datetime(dat['released_on'], format='%Y-%m-%d').dt.year, bins=10, labels=['1','2','3','4','5','6','7','8','9','10'])
dat['year_q_bin'] = pd.qcut(pd.to_datetime(dat['released_on'], format='%Y-%m-%d').dt.year, q=5, duplicates='drop',
                            labels=list(map(str, range(1, 6))))

dat['franchise'] = np.where(dat['franchise']==1.0, 'Yes', 'No')

C:\Users\Alex\AppData\Local\Programs\Python\Python38\lib\site-packages\easymoney\money.py:213: UserWarning: 
Inflation (CPI) data for 2020 in 'United States' could not be obtained from the
International Monetary Fund database currently cached.
Falling back to 2019.
  warn(warn_msg % (year, natural_region_name, str(fall_back_year)))


Add production company bins


In [86]:
prod = pd.read_csv('../extras/production_company_info.csv').loc[:, 'production_company':]

In [87]:
prod['box_office_tot'] = [int(re.sub('[$]|[,]', '', str(x))) for x in prod['box_office_tot']]

In [88]:
prod = prod.sort_values('box_office_tot', ascending=False)

prod['rank'] = list(range(1, len(prod)+1))
prod = prod.set_index('production_company')

In [89]:
dat = dat.join(prod, on='production_company').drop(['movie_count', 'box_office_tot', 'world_box_office_tot'], axis=1)

In [90]:
dat['production_company_bin'] = pd.cut(dat['rank'], bins=[-np.inf, 5, 100, np.inf], labels=['Top5', 'Top100', 'Top100+'])

In [91]:
dat = dat.rename(columns={'brand_ods_id_x': 'brand_ods_id'})

In [92]:
dat.to_csv('F:/capstone_listenfirst/brands_with_extras_kmodes.csv', index=False)

In [93]:
dat.to_csv('../data/brands_with_extras_kmodes.csv', index=False)